In [1]:
import VASPsol as vs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import os
ef = vs.ef
os.getcwd()

'/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks'

In [2]:
client, cluster = ef.dask_workers(32, 6, 10, burst=False)
client

NameError: name 'ef' is not defined

In [11]:
# get all dirs with a "VAC" folder in them
root_dir = '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/'
dirs = [os.path.abspath(root_dir+i) for i in next(os.walk(root_dir))[1] if 'VAC' in os.listdir(root_dir+i)]
data_jobs = [client.submit(vs.data, i) for i in dirs]
len(data_jobs)

321

In [12]:
# get all folders in root_dir
dirs = [os.path.abspath(i) for i in next(os.walk('.'))[1]]
dirs

['/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/models',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/figures']

In [13]:
from dask.distributed import progress
progress(data_jobs)

VBox()

In [14]:
ml_df = pd.concat([i.result().ml_df for i in data_jobs])
ml_df['error'] = ml_df['error'].abs()
ml_df = ml_df[ml_df['error'] < 10]
ml_df

,Solvent,FileHandle,directory,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,SoluteName,...,OP,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac
0,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.003000,0.642036,0.000772,-0.194341,-62.879039,2179,ethylformate,...,0.0,0.0,0.0,0.0,0.0,0.0,137.183,1.831503,0.079423,-0.691133
1,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.003660,0.581300,0.000936,-0.187315,-62.872013,2179,ethylformate,...,0.0,0.0,0.0,0.0,0.0,0.0,137.183,1.669484,0.072397,-0.629994
2,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002500,0.600000,0.000525,-0.177813,-62.862511,2179,ethylformate,...,0.0,0.0,0.0,0.0,0.0,0.0,137.183,1.450368,0.062895,-0.547309
3,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002750,0.660000,0.000472,-0.227300,-62.911998,2179,ethylformate,...,0.0,0.0,0.0,0.0,0.0,0.0,137.183,2.591538,0.112382,-0.977939
4,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002997,0.648200,0.000854,-0.185208,-62.869906,2179,ethylformate,...,0.0,0.0,0.0,0.0,0.0,0.0,137.183,1.620896,0.070290,-0.611659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,water,0100met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002500,0.600000,0.000525,-0.138300,-129.040770,2187,methylhexanoate,...,0.0,0.0,0.0,0.0,0.0,0.0,233.686,0.699198,0.030321,-0.280802
2,water,0100met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002997,0.648200,0.000854,-0.127730,-129.030200,2187,methylhexanoate,...,0.0,0.0,0.0,0.0,0.0,0.0,233.686,0.455454,0.019751,-0.182913
0,water,0010met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.003660,0.581300,0.000936,0.067036,-73.528551,2101,2-methylpropane,...,0.0,0.0,0.0,0.0,0.0,0.0,143.412,0.774150,0.033571,0.333685
1,water,0010met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.002500,0.600000,0.000525,0.037179,-73.558408,2101,2-methylpropane,...,0.0,0.0,0.0,0.0,0.0,0.0,143.412,1.462652,0.063428,0.630454


In [15]:
grouped_df = ml_df.groupby(['NC_K', 'SIGMA_K','TAU'])
# get only groups with at least 10 data points
grouped_df = grouped_df.filter(lambda x: len(x) > 10)
grouped_df = grouped_df.groupby(['NC_K', 'SIGMA_K','TAU'])
grouped_df = grouped_df.mean()
# include the count of data points in the group
grouped_df['count'] = ml_df.groupby(['NC_K', 'SIGMA_K','TAU']).count()['error']
grouped_df = grouped_df.reset_index()
grouped_df

/tmp/ipykernel_15646/2260228251.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = grouped_df.mean()


,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,Charge,Level1,Level2,Level3,...,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac,count
0,0.002500,0.600000,0.000525,-0.120907,-79.148891,2220.932000,0.0,4.444000,2.340000,0.848000,...,3.055388,0.263200,0.0,0.725388,0.312784,165.952500,1.179053,0.001062,-0.038961,250
1,0.002750,0.660000,0.000472,-0.169862,-81.834564,2211.274882,0.0,4.004739,2.464455,0.966825,...,2.996616,0.311559,0.0,0.498777,0.370597,166.653005,1.506551,0.043645,-0.258845,211
2,0.002997,0.648200,0.000854,-0.116700,-82.273096,2212.088496,0.0,4.017699,2.486726,0.946903,...,3.059314,0.291150,0.0,0.572801,0.346000,167.244765,1.101946,-0.007575,-0.258933,226
3,0.003000,0.642036,0.000772,-0.127350,-84.674760,2210.926966,0.0,3.955056,2.426966,1.011236,...,2.878584,0.181646,0.0,0.735489,0.233680,170.553685,1.118842,0.003082,-0.198540,178
4,0.003087,0.591554,0.000790,-0.139768,-93.893031,2201.000000,0.0,3.555556,2.333333,1.666667,...,1.921222,0.000000,0.0,1.374278,0.000000,181.261222,1.143174,-0.026103,8.996397,18
5,0.003660,0.581300,0.000936,-0.116438,-81.996421,2214.160870,0.0,4.147826,2.452174,0.926087,...,3.275357,0.286100,0.0,0.750530,0.339983,168.009948,1.120684,-0.007910,-0.207903,230


In [16]:
# compute the % increase in performance for each group
grouped_df['% increase'] = (grouped_df['error'] - grouped_df['error'].min()) / grouped_df['error'].min() * 100
grouped_df

,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,Charge,Level1,Level2,Level3,...,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac,count,% increase
0,0.002500,0.600000,0.000525,-0.120907,-79.148891,2220.932000,0.0,4.444000,2.340000,0.848000,...,0.263200,0.0,0.725388,0.312784,165.952500,1.179053,0.001062,-0.038961,250,6.997376
1,0.002750,0.660000,0.000472,-0.169862,-81.834564,2211.274882,0.0,4.004739,2.464455,0.966825,...,0.311559,0.0,0.498777,0.370597,166.653005,1.506551,0.043645,-0.258845,211,36.717265
2,0.002997,0.648200,0.000854,-0.116700,-82.273096,2212.088496,0.0,4.017699,2.486726,0.946903,...,0.291150,0.0,0.572801,0.346000,167.244765,1.101946,-0.007575,-0.258933,226,0.000000
3,0.003000,0.642036,0.000772,-0.127350,-84.674760,2210.926966,0.0,3.955056,2.426966,1.011236,...,0.181646,0.0,0.735489,0.233680,170.553685,1.118842,0.003082,-0.198540,178,1.533265
4,0.003087,0.591554,0.000790,-0.139768,-93.893031,2201.000000,0.0,3.555556,2.333333,1.666667,...,0.000000,0.0,1.374278,0.000000,181.261222,1.143174,-0.026103,8.996397,18,3.741336
5,0.003660,0.581300,0.000936,-0.116438,-81.996421,2214.160870,0.0,4.147826,2.452174,0.926087,...,0.286100,0.0,0.750530,0.339983,168.009948,1.120684,-0.007910,-0.207903,230,1.700421


In [17]:
# parameters from optimizer
#NC_K_opt = 0.002750
#SIGMA_K_opt = 0.660000
#TAU_opt = 0.000472
# NC_K_opt = 3.66e-3
# SIGMA_K_opt = 0.5813
# TAU_opt =9.363e-4
    
NC_K_opt = 2.997e-03
SIGMA_K_opt = 6.482e-01
TAU_opt = 8.541e-04
# new NC_K_opt, etc = 0.00308742, 0.59155393, 0.00079023
# NC_K_opt = 0.00308742
# SIGMA_K_opt = 0.59155393
# TAU_opt = 0.00079023
NC_K_default = 0.0025
SIGMA_K_default = 0.6
TAU_default = 0.000525
# make opt = default
NC_K_opt = NC_K_default
SIGMA_K_opt = SIGMA_K_default
TAU_opt = TAU_default


In [18]:
default_df = ml_df[(ml_df['NC_K'] == NC_K_default) & (ml_df['SIGMA_K'] == SIGMA_K_default) & (ml_df['TAU'] == TAU_default)]
default_df['error'] = default_df['error'].abs()
default_df['error'].mean(), len(default_df)

/tmp/ipykernel_15646/765186932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  default_df['error'] = default_df['error'].abs()


(1.1790533854799914, 250)

In [19]:
# get existing data that was ran with the above parameters
ran_df = ml_df[(ml_df['NC_K'] == NC_K_opt) & (ml_df['SIGMA_K'] == SIGMA_K_opt) & (ml_df['TAU'] == TAU_opt)]
ran_df = ran_df[ran_df['FileHandle'].isin(default_df['FileHandle'])]
ran_df = ran_df.dropna()
ran_df = ran_df.reset_index(drop=True)
ran_df['error'] = ran_df['error'].abs()
ran_df = ran_df[ran_df['error'] < 10]
ran_df

,Solvent,FileHandle,directory,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,SoluteName,...,OP,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac
0,water,0092ethb,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.177813,-62.862511,2179,ethylformate,...,0.000,0.000,0.000,0.0,0.000,0.0,137.183,1.450368,0.062895,-0.547309
1,water,0145pro,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.190273,-55.219571,2231,allylalcohol,...,0.000,0.000,0.000,0.0,0.000,0.0,125.517,0.692305,-0.030022,0.136280
2,water,0238met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.126110,-162.077690,2302,methyloctanoate,...,0.000,0.000,0.000,0.0,0.000,0.0,282.436,0.868097,0.037645,-0.425538
3,water,0007nhe,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,0.060560,-123.105020,2099,n-heptane,...,0.000,0.000,0.000,0.0,0.000,0.0,219.446,1.223486,0.053057,0.466980
4,water,0067but,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.054671,-96.262839,2155,t-butylmethylether,...,0.000,0.000,0.000,0.0,0.000,0.0,171.704,0.949287,-0.041166,0.429542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,water,0217wat,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.310633,-14.529120,2285,water,...,0.000,0.000,0.000,0.0,0.000,0.0,55.683,0.853197,0.036999,-0.135213
246,water,0449pho,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.333980,-218.388970,2346,ethyl4-cyanophenylphenylthiophosphonate,...,10.036,33.954,0.000,0.0,24.211,0.0,378.215,2.601579,0.112818,-0.510113
247,water,0142die,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.051419,-77.307156,2228,diethylsulfide,...,0.000,25.165,0.061,0.0,0.000,0.0,172.471,0.244278,-0.010593,0.170824
248,water,0100met,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.138300,-129.040770,2187,methylhexanoate,...,0.000,0.000,0.000,0.0,0.000,0.0,233.686,0.699198,0.030321,-0.280802


In [20]:
# get the mean absolute error for ran_df 
error = ran_df['error'].mean()
print(error)
# sort by the error
sorted_df = ran_df.sort_values(by=['error'], ascending=False)
sorted_df

1.1790533854799914


,Solvent,FileHandle,directory,NC_K,SIGMA_K,TAU,Solvation_Energy,Total_Energy,No.,SoluteName,...,OP,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac
216,water,0114NNd,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.056840,-121.977140,2200,"N,N'-dimethylpiperazine",...,0.0,0.0,0.0,0.0,0.0,0.0,198.625,6.269270,-0.271868,0.827080
211,water,0048pro,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,0.029162,-63.504602,2137,isopropanol,...,0.0,0.0,0.0,0.0,0.0,0.0,132.546,5.432476,-0.235580,1.141276
140,water,0112Nme,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.108180,-105.652610,2198,N-methylpiperazine,...,0.0,0.0,0.0,0.0,0.0,0.0,177.663,5.275369,-0.228767,0.678941
115,water,0227Nme,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.100520,-100.155180,2293,N-methylmorpholine,...,0.0,0.0,0.0,0.0,0.0,0.0,172.561,4.022009,-0.174415,0.634386
57,water,0225pipa,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.048894,-94.190572,2292,piperidine,...,0.0,0.0,0.0,0.0,0.0,0.0,161.954,3.982504,-0.172702,0.779355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,water,0018cyc,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,0.051519,-99.037092,2108,cyclohexane,...,0.0,0.0,0.0,0.0,0.0,0.0,167.066,0.041972,0.001820,0.034123
160,water,0090hex,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.267790,-113.072470,2177,hexanoicacid,...,0.0,0.0,0.0,0.0,0.0,0.0,208.827,0.034763,-0.001507,0.005598
48,water,0038oxy,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.037850,-109.293890,2128,o-xylene,...,0.0,0.0,0.0,0.0,0.0,0.0,184.859,0.027179,-0.001179,0.030199
65,water,0085non,/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_...,0.0025,0.6,0.000525,-0.116760,-155.205880,2172,5-nonanone,...,0.0,0.0,0.0,0.0,0.0,0.0,269.167,0.022486,0.000975,-0.008422


In [21]:
def solv_calc(file, nc, sig, tau):
    # This is where you can change the solvent that is being run~!!!
    return vs.calculate_solvent(file, NSW=0, solvent='water', calc_type='non-linear', dict_of_additional_tags={'NC_K':nc, 'SIGMA_K':sig, 'TAU':tau})


In [22]:
ran_files = [os.path.abspath(i) for i in sorted_df['FileHandle']]
ran_files

['/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0114NNd',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0048pro',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0112Nme',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0227Nme',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0225pipa',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0109pip',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0105aze',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0149mor',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0406oct',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0108pyr',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0405hex',
 '/blue/hennig/ericf

In [23]:
files_to_run = [i for i in dirs if i not in ran_files]
# we are going to use rejection sampling to get 20 random files to run
# we will use the error as the probability of being selected
def rejection_sampling(vector, samples=100):
    indicies = np.arange(len(vector))
    # fit a gaussian kernel to the data
    from sklearn.neighbors import KernelDensity
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(vector.reshape(-1, 1))
    # create a vector of each points probability
    logprob = kde.score_samples(vector.reshape(-1, 1))
    p = np.exp(logprob)
    # normalize the probabilities
    p = p / p.sum()

    output = []
    for sample in range(samples):
        # use the probability to sample the data
        sample = np.random.choice(indicies, p=p)
        # add the sample to the vector
        output.append(sample)
    return output
# use the df to get the error and find the corresponding run files

# UNCOMMENT LATER
# error = sorted_df['error'].values
# # get the indicies of the files to run
# indicies = rejection_sampling(error, samples=20)
# # get the files to run
# fnames_to_run = sorted_df['FileHandle'].iloc[indicies].values
# #print(len(fnames_to_run), fnames_to_run)
# # get the full path to the files
# files_to_run = [os.path.abspath(i) for i in fnames_to_run]

files_to_run = ['/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0036tol', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0221tri', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0046eth', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0029hex', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0176pdi', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0033pen', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0176pdi', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0234ENmb', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0571dim', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0401amia', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0071proa', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0063die', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0050met', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0078pen', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0093met', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0110but', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0068ani', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0076but', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0034hex', 
                '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0438pho']
fnames_to_run = [os.path.basename(i) for i in files_to_run]
print(len(files_to_run), files_to_run, fnames_to_run)

20 ['/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0036tol', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0221tri', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0046eth', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0029hex', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0176pdi', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0033pen', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0176pdi', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0234ENmb', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0571dim', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/0401amia', '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/calculations/007

In [24]:
ml_df[(ml_df['FileHandle'].isin(fnames_to_run))].groupby(['NC_K','SIGMA_K','TAU']).mean()


/tmp/ipykernel_15646/1507153532.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ml_df[(ml_df['FileHandle'].isin(fnames_to_run))].groupby(['NC_K','SIGMA_K','TAU']).mean()


,,,Solvation_Energy,Total_Energy,No.,Charge,Level1,Level2,Level3,DeltaGsolv,eps,n,...,OP,S,HS,OS,SP,SS,TotalArea,error,error_ev,error_frac
NC_K,SIGMA_K,TAU,,,,,,,,,,,,,,,,,,,,,
0.002500,0.600000,0.000525,-0.151376,-91.898562,2198.736842,0.0,3.473684,2.421053,1.631579,-3.804737,78.36,1.3328,...,2.691474,1.820105,0.0,0.0,1.301947,0.0,178.262684,1.045805,-0.013617,10.093717
0.002750,0.660000,0.000472,-0.206204,-93.959467,2201.000000,0.0,3.555556,2.333333,1.666667,-3.825000,78.36,1.3328,...,2.841000,1.921222,0.0,0.0,1.374278,0.0,181.261222,1.228921,0.040333,15.926095
0.002997,0.648200,0.000854,-0.149138,-91.896324,2198.736842,0.0,3.473684,2.421053,1.631579,-3.804737,78.36,1.3328,...,2.691474,1.820105,0.0,0.0,1.301947,0.0,178.262684,0.972275,-0.015855,9.171095
0.003000,0.642036,0.000772,-0.161197,-93.914460,2201.000000,0.0,3.555556,2.333333,1.666667,-3.825000,78.36,1.3328,...,2.841000,1.921222,0.0,0.0,1.374278,0.0,181.261222,0.922263,-0.004674,11.033503
0.003087,0.591554,0.000790,-0.139768,-93.893031,2201.000000,0.0,3.555556,2.333333,1.666667,-3.825000,78.36,1.3328,...,2.841000,1.921222,0.0,0.0,1.374278,0.0,181.261222,1.143174,-0.026103,8.996397
0.003660,0.581300,0.000936,-0.148650,-93.901913,2201.000000,0.0,3.555556,2.333333,1.666667,-3.825000,78.36,1.3328,...,2.841000,1.921222,0.0,0.0,1.374278,0.0,181.261222,1.007832,-0.017222,9.567164


In [25]:
jobs = [client.submit(solv_calc, os.path.abspath('../calculations/'+i), NC_K_opt, SIGMA_K_opt, TAU_opt) for i in ml_df['FileHandle'].unique()]


In [31]:
jobs = [client.submit(solv_calc, os.path.abspath('../calculations/'+i.split('/')[-1]), NC_K_opt, SIGMA_K_opt, TAU_opt) for i in ran_files]

In [30]:
ran_files

['/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0437pho',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0115dip',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0107tri',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0066dim',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0042ant',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0125dim',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0203bro',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0571dim',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0401amia',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0032but',
 '/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/VaspPysol/notebooks/0034hex',
 '/blue/hennig/ericf

In [1]:
from dask.distributed import progress
progress(jobs)

NameError: name 'jobs' is not defined

In [1]:
# cancel all the jobs
for job in jobs:
    job.cancel()

NameError: name 'jobs' is not defined

In [ ]:
data = [vs.data(i) for i in dirs]
data

/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0092ethb/Solvent_water_epsilon_78.4_Ionic_0_calc_linear_NC_K_0.00366_SIGMA_K_0.5813_TAU_0.0009363
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0092ethb/Solvent_water_epsilon_78.4_Ionic_0_calc_linear
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0092ethb/Solvent_water_epsilon_78.4_Ionic_0_calc_linear_NC_K_0.00275_SIGMA_K_0.66_TAU_0.000472
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0145pro/Solvent_water_epsilon_78.4_Ionic_0_calc_linear_NC_K_0.00366_SIGMA_K_0.5813_TAU_0.0009363
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0145pro/Solvent_water_epsilon_78.4_Ionic_0_calc_linear
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0145pro/Solvent_water_epsilon_78.4_Ionic_0_calc_linear_NC_K_0.00275_SIGMA_K_0.66_TAU_0.000472
/blue/hennig/ericfonseca/NASA/VASPsol/Truhlar_Benchmarks/EMA/0577hyd/Solvent_water_epsilon_78.4_Ionic_0_calc_linear_NC_K_0.00366_SIGMA_K_0.

KeyboardInterrupt: 

In [2]:
run_data = [vs.data(i) for i in files_to_run]
run_df = pd.concat([i.ml_df for i in run_data])
run_df

NameError: name 'files_to_run' is not defined

In [ ]:
# group by vaspsol parameters
run_df['error'] = run_df['error'].abs()
grouped_runs = run_df.groupby(['NC_K', 'SIGMA_K', 'TAU'])
# mean error
mean_error = grouped_runs['error'].mean()
mean_error

NC_K     SIGMA_K   TAU     
0.00250  0.600000  0.000525    1.006504
0.00275  0.660000  0.000472    4.614736
0.00300  0.642036  0.000772    7.896860
0.00366  0.581300  0.000936    0.968037
Name: error, dtype: float64

In [3]:
#ml_df = pd.concat([i.ml_df for i in data])
ml_df = ml_df[(ml_df['NC_K'] == NC_K_opt) & (ml_df['SIGMA_K'] == SIGMA_K_opt) & (ml_df['TAU'] == TAU_opt)]
ml_df = ml_df.dropna()
ml_df = ml_df.reset_index(drop=True)
ml_df['error'] = ml_df['error'].abs()
ml_df = ml_df[ml_df['error'] < 10]
ml_df

NameError: name 'ml_df' is not defined

In [4]:
ml_df.hist('error', bins=20)

NameError: name 'ml_df' is not defined

In [5]:
mean_error = ml_df['error'].mean()
mean_error, 

NameError: name 'ml_df' is not defined

In [ ]:
names = ['n-octane','ethanol','nitromethane','phenol','toluene','1,4-dioxane','acetone','aceticacid','2-butanone']

In [6]:
df2 = ml_df[ml_df['SoluteName'].isin(names)]
df2

NameError: name 'ml_df' is not defined

In [7]:
df2['error'].mean()

NameError: name 'df2' is not defined

In [8]:
client

NameError: name 'client' is not defined